In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
import tensorflow as tf
from video.data import read_dataset

In [3]:
tf.enable_eager_execution()

In [4]:
!gsutil cp gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv .

Copying gs://asl-mixi-project-bucket/data/youtube-8m/vocabulary.csv...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      


In [5]:
import csv

label_map = {}
with open('./vocabulary.csv', 'r') as c:
    reader = csv.reader(c)
    next(reader, None)
    for r in reader:
        label_map[r[0]] = r[3]

In [6]:
data = read_dataset(
    'gs://asl-mixi-project-bucket/data/youtube-8m/valid/*.tfrecord',
    tf.estimator.ModeKeys.EVAL,
    batch_size=5
).shuffle(1024)
batch, labels = next(iter(data))
batch.pop('id')
feature = [{'mean_rgb': r.numpy().tolist(), 'mean_audio': a.numpy().tolist()} for r, a in zip(batch['mean_rgb'], batch['mean_audio'])]
labels = [[label_map[str(i)] for i, v in enumerate(label) if v] for label in labels]
labels

[['Musician',
  'Guitar',
  'String instrument',
  'Musical ensemble',
  'Drum kit',
  'Drum'],
 ['Game', 'Association football'],
 ['Performance art', 'Dance'],
 ['Skateboarding', 'Surfing', 'Surfboard', 'Bodyboarding'],
 ['Game', 'Video game', 'Silkroad Online']]

In [7]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

PROJECT = 'qwiklabs-gcp-ml-83914069970b'
MODEL_NAME = 'youtube8mkojo'
VERSION = 'v1'

credentials = GoogleCredentials.get_application_default()
api = discovery.build(
    "ml",
    "v1",
    credentials = credentials,
    discoveryServiceUrl = "https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json"
)

def show_predict(prediction, label):
    print(label)
    for i, index in enumerate(prediction["predicted_topk"]):
        print("{}th({}:{}) probality is {}".format(
            (i+1),
            index,
            label_map["{}".format(index)],
            prediction['probabilities'][index]
        ))
    print('')

def predict(model_name=MODEL_NAME, version=VERSION):
    request_data = {"instances": feature}
    parent = "projects/{}/models/{}/versions/{}".format(PROJECT,MODEL_NAME, VERSION)
    response = api.projects().predict(body = request_data, name = parent).execute()
    for p, l in zip(response['predictions'], labels):
        show_predict(p, l)
    return None

In [8]:
predict('youtube8mkojo', 'v1')

['Musician', 'Guitar', 'String instrument', 'Musical ensemble', 'Drum kit', 'Drum']
1th(4:Musician) probality is 0.9946284294128418
2th(9:Guitar) probality is 0.9855870604515076
3th(10:String instrument) probality is 0.8845663070678711
4th(13:Musical ensemble) probality is 0.768027663230896
5th(33:Drum) probality is 0.7544289827346802

['Game', 'Association football']
1th(12:Association football) probality is 0.9842694401741028
2th(0:Game) probality is 0.9665849208831787
3th(91:Highlight film) probality is 0.06655606627464294
4th(237:Stadium) probality is 0.039057791233062744
5th(312:Rugby football) probality is 0.014034807682037354

['Performance art', 'Dance']
1th(6:Performance art) probality is 0.9520819187164307
2th(8:Dance) probality is 0.8863418698310852
3th(3:Concert) probality is 0.013432294130325317
4th(812:Dance studio) probality is 0.011701822280883789
5th(206:Ballet) probality is 0.011263459920883179

['Skateboarding', 'Surfing', 'Surfboard', 'Bodyboarding']
1th(182:Surfing